In [50]:
import pandas as pd
from datetime import timedelta

# Примерные входные данные (замените своими)
# df_investments: date, amount
df_investments = pd.DataFrame({
    'date': pd.date_range('2020-01-01', periods=36, freq='MS'),
    'amount': [1000]*36
})

# df_rates: date, rate (где rate - годовая ставка, например 0.06 для 6%)
df_rates = pd.DataFrame({
    'date': pd.date_range('2020-01-01', periods=36, freq='MS'),
    'rate': [0.06, 0.06, 0.06] * 12
    #'rate': [0.06, 0.05, 0.07] * 12
})

# Параметры
n = 1000  # минимальная сумма для открытия вклада
investment_duration = 12  # продолжительность вклада в месяцах

# Объединяем данные по датам
df = pd.merge(df_investments, df_rates, on='date', how='left')

# Рассчитаем доходность по вкладам
def calculate_income(df, min_sum, duration):
    total_income = []
    balance = 0  # текущий баланс для инвестирования
    start_dates = []

    for i, row in df.iterrows():
        balance += row['amount']
        if balance >= min_sum:
            # Открываем вклад
            start_date = row['date']
            end_date = start_date + pd.DateOffset(months=duration)
            deposit_rate = df[(df['date'] == start_date)]['rate'].mean()
            # deposit_rate = df[(df['date'] >= start_date) & (df['date'] < end_date)]['rate'].mean()
            print("deposit_rate", deposit_rate)
            
            if pd.notna(deposit_rate):
                income = balance * deposit_rate * (duration / 12)
                total_income.append({
                    'start_date': start_date,
                    'end_date': end_date,
                    'principal': balance,
                    'rate': deposit_rate,
                    'income': income
                })
            
            # Обнуляем баланс после открытия вклада
            balance = 0
    
    return pd.DataFrame(total_income)

# Вычисление доходности первичных вкладов
df_income = calculate_income(df, n, investment_duration)
print(df_income)


deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
deposit_rate 0.06
   start_date   end_date  principal  rate  income
0  2020-01-01 2021-01-01       1000  0.06    60.0
1  2020-02-01 2021-02-01       1000  0.06    60.0
2  2020-03-01 2021-03-01       1000  0.06    60.0
3  2020-04-01 2021-04-01       1000  0.06    60.0
4  2020-05-01 2021-05-01       1000  0.06    60.0
5  2020-06-01 2021-06-01       1000  0.06    60.0
6 

In [51]:
max_date = df['date'].max()
max_date

Timestamp('2022-12-01 00:00:00')

In [52]:
from dateutil.relativedelta import relativedelta
from datetime import datetime

# Обработка первичных вкладов.
# Убираю периоды "будущей доходности".
max_date = df['date'].max()
correct_income = []
for i, row in df_income.iterrows():
    if max_date < row["end_date"]:
        # n_month = relativedelta(max_date, )
        n_month = (max_date.year - row["start_date"].year) * 12 + (max_date.month - row["start_date"].month)
        row["income"] = n_month / 12 * row["income"]
        
        correct_income.append(row)
    else:
        correct_income.append(row)
        

In [53]:
df_income_correct = pd.DataFrame(correct_income)

# Расчёт повторных инвестиций.

In [54]:
income_new_all = []
for i, row in df_income_correct.iterrows():
    #row["end_date"]
    income_all_reinvest = []
    end_date_iter = row['end_date']
    total_money = row["principal"] + row["income"]
    while True:
        if end_date_iter <= max_date:
            n_month_for_end = (max_date.year - end_date_iter.year) * 12 + (max_date.month - end_date_iter.month)
            n_month_deposit = min([12, n_month_for_end]) # Сколько месяцев вклада произошло.
            rate_in_date = df_rates.loc[df_rates.date == end_date_iter, "rate"].mean()
            new_income = (rate_in_date * total_money) * (n_month_deposit / 12)
            
            total_money += new_income
            end_date_iter = end_date_iter + pd.DateOffset(months=12)
            income_all_reinvest.append(new_income)
        else:
            break    
    
    income_new_all.append(income_all_reinvest)
    # break

In [55]:
df_income_new_all = pd.DataFrame(income_new_all)

In [56]:
i = 2
for col in list(df_income_new_all):
     col_name = "income_" + str(i)
     i += 1
     df_income_correct[col_name] = df_income_new_all[col]

In [57]:
df_income_correct[['principal', 'income', 'income_2', 'income_3']].sum().sum()

39253.788

In [58]:
df_income_correct

,start_date,end_date,principal,rate,income,income_2,income_3
0,2020-01-01,2021-01-01,1000,0.06,60.0,63.6,61.798
1,2020-02-01,2021-02-01,1000,0.06,60.0,63.6,56.180
2,2020-03-01,2021-03-01,1000,0.06,60.0,63.6,50.562
3,2020-04-01,2021-04-01,1000,0.06,60.0,63.6,44.944
4,2020-05-01,2021-05-01,1000,0.06,60.0,63.6,39.326
5,2020-06-01,2021-06-01,1000,0.06,60.0,63.6,33.708
6,2020-07-01,2021-07-01,1000,0.06,60.0,63.6,28.090
7,2020-08-01,2021-08-01,1000,0.06,60.0,63.6,22.472
8,2020-09-01,2021-09-01,1000,0.06,60.0,63.6,16.854
9,2020-10-01,2021-10-01,1000,0.06,60.0,63.6,11.236


In [59]:
df_income[["principal", "income"]].sum().sum()

38160.0

In [61]:
max_date

Timestamp('2022-12-01 00:00:00')